- Load the Iris dataset from scikit-learn. The third column represents the petal length, and the fourth column the petal width of the flower samples. The classes are already converted to integer labels where 0=Iris-Setosa, 1=Iris-Versicolor, 2=Iris-Virginica.
- Train different classifiers using off-the-shelf algorithms in scikit-learn.

3